# 3DOF Robot 

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import time 
import modern_robotics as mr 
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
client=RemoteAPIClient()

sim=client.getObject("sim")
base_handle=sim.getObject("/Base")
ee_handle=sim.getObject("/EE_Frame")
joint_names=["/joint1","/joint2","/joint3"]
joint_handles=[sim.getObject(k)for k in joint_names]
#Extract the M 
for j in joint_handles:
    sim.setJointPosition(j,0)
time.sleep(0.5)
t_raw=sim.getObjectMatrix(ee_handle,base_handle)
M=np.array([
        [t_raw[0],t_raw[1],t_raw[2],t_raw[3]],
        [t_raw[4],t_raw[5],t_raw[6],t_raw[7]],
        [t_raw[8],t_raw[9],t_raw[10],t_raw[11]],
        [0,0,0,1]
    ])
#print(M)

#Extract Slist 
def get_screw_axis(joint_handles,base_handle):
    slist=[]
    for j in joint_handles:
        s=sim.getObjectMatrix(j,base_handle)
        s_list=np.array([
        [s[0],s[1],s[2]],
        [s[4],s[5],s[6]],
        [s[8],s[9],s[10]]
    ])
        w=[s[2],s[6],s[10]]
        q=[s[3],s[7],s[11]]
        v=-np.cross(w,q)
        S=np.hstack((w,v))
        slist.append(S)
    Slist=np.array(slist).T
    return Slist 
slist=get_screw_axis(joint_handles,base_handle)
#print(slist)

#Apply the Foward Kinematics
#What forward kinematics does?
#it takes the home configuration of the end-effector and the screw axes of the joints, and the joint angles, and compute the current configuration of the end-effector in space.
theta=np.array([1.1,0.5,1.7])   #i take angle for three joints in radians
T=mr.FKinSpace(M,slist,theta) 
print(f"Current configuration of the end-effector:\n{T}")

#Above, i calculated the theoretical end-effector configuration using the forward kinematics.
#Now i will compare it with the acutal end effector configuration obtained from the simulation.

#Move the robot in the simulation to the desired joint angles 
for a,b in zip(joint_names,theta):
    h=sim.getObject(a)
    sim.setJointPosition(h,b)
time.sleep(2.5)
for _ in range(20):
    sim.step()
    time.sleep(0.01)
#Read the actual end_effector configuration from the simulation
t=sim.getObjectMatrix(ee_handle,base_handle)
T_actual=np.array([
    [t[0],t[1],t[2],t[3]],
    [t[4],t[5],t[6],t[7]],
    [t[8],t[9],t[10],t[11]],
    [0,0,0,1]
])
#Now we calculate the error between T(theoretical) and T_actual(simulation)
error=np.linalg.norm(T-T_actual)
print(error)
if error<0.01:
    print("Forward kinematics is correct")
else:
    print("Forward kinematics is incorrect")

Current configuration of the end-effector:
[[ 0.47158104 -0.0189218  -0.8816197  -0.07318809]
 [-0.72742265  0.55678958 -0.40105069 -0.34282129]
 [ 0.49846526  0.83043803  0.24880726  0.84695659]
 [ 0.          0.          0.          1.        ]]
1.0011263372118013e-15
Forward kinematics is correct


In [3]:
#Previously we used base frame ,now we will use the end-effector frame as the reference frame 
angles=np.array([1.1,1.5,0.7])
Blist=mr.Adjoint(np.linalg.inv(M))@slist    #The M matrix has information about the End-effector frame.
T_calculated=mr.FKinBody(M,Blist,angles)
print(T_calculated)
#Move the robot in simulation to the desired joint angles
for o,p in zip(joint_handles,angles):
    sim.setJointPosition(o,p)
time.sleep(2.5)
for _ in range(20):
    sim.step()
    time.sleep(0.01)
#We read the actual end-effector configuration from the simulation 
t=sim.getObjectMatrix(ee_handle,base_handle)
T_real=np.array([
    [t[0],t[1],t[2],t[3]],
    [t[4],t[5],t[6],t[7]],
    [t[8],t[9],t[10],t[11]],
    [0,0,0,1]
])
#Find the error between T_calculated and T_real
error__body=np.linalg.norm(T_calculated-T_real)
print(error__body)
if error__body<0.01:
    print("Forward kinematics is correct")
else:
    print("Forward kinematics is incorrect")

[[ 0.61555296  0.78434135 -0.07683229  0.42068497]
 [-0.41340536  0.23835668 -0.87879583 -0.4372566 ]
 [-0.67096242  0.57270825  0.47097208  0.36894702]
 [ 0.          0.          0.          1.        ]]
1.6895031135148352e-15
Forward kinematics is correct


### Jacobian 

In [4]:
theta_angle=np.array([2.1,0.4,0.1])
for q,r in zip(joint_handles,theta_angle):
    sim.setJointPosition(q,r)
time.sleep(2.5)
for _ in range(20):
    sim.step()
    time.sleep(0.01)
#jacobian in space frame
js_in_space=mr.JacobianSpace(slist,theta_angle)
print(js_in_space)                                  #Here we got the 6 by 3 matrix ,which mean at present pose 
                                                    #If we consider joint 1 and 2 velocities equal to zero.Only Joint3
                                                    #rotate with 1 rad/s then what changes occur in End_Effector,column3
                                                    #represent these changes .Columns 3 has six rows ,first three for 
                                                    #angular velocity  change ,last three for linear velocity change .
                                                    #Same concept for Joint 1(column 1) and joint 2 (columns 2).So in
                                                    #this way 6 by 3 matrix complete .We cal jacobian in body or space
                                                    #frame both .Then this jacobian is used for velocity mapping ,joint
                                                    #velocity to the end effector velocity.
#jacobian in body frame
js_in_body=mr.JacobianBody(Blist,theta_angle)
#print(js_in_body)

[[ 1.00000000e+00  3.48994967e-02 -9.20083977e-01]
 [ 0.00000000e+00 -5.04538566e-01 -3.57641717e-01]
 [ 0.00000000e+00  8.62683523e-01 -1.59805750e-01]
 [ 0.00000000e+00  1.65261067e-13  2.03002046e-01]
 [-1.22124533e-15 -1.15859764e-01 -3.91730377e-01]
 [-0.00000000e+00 -6.77603289e-02 -2.92103413e-01]]


### Applications of Jacobian 

#### 1:Singularities Detection 

In [31]:
test_config=np.array([2.1,0.4,-3.14])
for a,b in zip(joint_handles,test_config):
    sim.setJointPosition(a,b)
time.sleep(0.01)
for _ in range(20):
    sim.step()
    time.sleep(0.5)
jacobian=mr.JacobianSpace(slist,test_config)
#Compute the rank of jacobian matrix 
rank=np.linalg.matrix_rank(jacobian,tol=1e-4)
if rank==3:
    print("No Singularity")
else:
    print("Singularity Detect")
print(rank)

# YOSHIKAWA MANIPULABILITY
A=jacobian.T@jacobian
w=np.sqrt(np.linalg.det(A))
print(w)

#Condition Numbers
condition_numbers=np.linalg.cond(jacobian)
print(condition_numbers)
if condition_numbers<10:
    print("No Singularity")
elif condition_numbers<100:
    print("Close to Singularity")
elif condition_numbers<1000:
    print("At Singularity")
else:
    print("singularity")

No Singularity
3
0.6550574810406399
3.200388311462933
No Singularity
